## Chat

### Load the index

In [ ]:
### Define the embedding
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
embed_model = HuggingFaceEmbedding(model_name="BAAI/bge-m3")

In [2]:
from llama_index.core import Settings, SimpleDirectoryReader
from llama_index.core import VectorStoreIndex, StorageContext, load_index_from_storage

documents = SimpleDirectoryReader(input_dir="../data/json/").load_data()

storage_path = "./chat_storage/"

# Persist into local disk
index = VectorStoreIndex.from_documents(
    documents,
    embed_model=embed_model
)
index.storage_context.persist(persist_dir=storage_path)

In [3]:
### Use Ollama
from llama_index.llms.ollama import Ollama
llm = Ollama(model="llama3.2:3b", request_timeout=600.0)

In [4]:
from llama_index.core.chat_engine.types import ChatMode
chat_engine = index.as_chat_engine(chat_mode=ChatMode.CONDENSE_PLUS_CONTEXT,llm=llm)

In [5]:
chat_engine

In [6]:
response = chat_engine.chat("NVIDIA 股價創新高有什麼影響？")

In [ ]:
response

Streaming Response

In [8]:
streaming_response = chat_engine.stream_chat("NVIDIA 股價創新高有什麼影響？")
for token in streaming_response.response_gen:
    print(token, end="")

根據文件中的資訊，NVIDIA 股價創新高將推動標準普爾 500 指數 (.SPX) 上漲 0.8%，創下新高。这意味著 NVIDIA 的股價上漲對標準普爾 500 指數的影響是非常顯著的。

Validate if the chat history really matters

In [9]:
streaming_response = chat_engine.stream_chat("台灣半導體公司會受到什麼影響嗎")
for token in streaming_response.response_gen:
    print(token, end="")

根據文件中的資訊，台湾半导体公司（2330.TW）作为NVIDIA的合同生产商，预计将于周四报告季度利润增长40%。这得益于需求激增。

In [10]:
for each in chat_engine.chat_history:
    print("Role: ", each.role)
    print("Content: ", each.content)
    print("-"*50)

Role:  MessageRole.USER
Content:  NVIDIA 股價創新高有什麼影響？
--------------------------------------------------
Role:  MessageRole.ASSISTANT
Content:  根據文件中的資訊，NVIDIA 股價創新高將使該公司的市值達到 3.39 兆美元，略低於 Apple 的 3.52 兆美元，且高於微軟的 3.12 兆美元。這意味著 NVIDIA 股價上漲對標準普爾 500 指數 (.SPX) 的影響將重大。

文件還提到，NVIDIA 在八月確認其即將推出的 Blackwell 芯片的生產提升將推遲至第四季，但公司淡化了此消息的影響。儘管如此，投資者對於 NVIDIA 的當前及下一代 AI 處理器的需求充滿信心，使該公司股價上漲並創新高。

另外，NVIDIA、Apple 和微軟合計約占標準普爾 500 指數的五分之一，因此其日常漲跌具有重大影響。
--------------------------------------------------
Role:  MessageRole.USER
Content:  NVIDIA 股價創新高有什麼影響？
--------------------------------------------------
Role:  MessageRole.ASSISTANT
Content:  根據文件中的資訊，NVIDIA 股價創新高將推動標準普爾 500 指數 (.SPX) 上漲 0.8%，創下新高。这意味著 NVIDIA 的股價上漲對標準普爾 500 指數的影響是非常顯著的。
--------------------------------------------------
Role:  MessageRole.USER
Content:  台灣半導體公司會受到什麼影響嗎
--------------------------------------------------
Role:  MessageRole.ASSISTANT
Content:  根據文件中的資訊，台湾半导体公司（2330.TW）作为NVIDIA的合同生产商，预计将于周四报告季度利润增长40%。这得益于需求激增。
------------

In [11]:
# Clear chat history
chat_engine.chat_history.clear()
print(chat_engine.chat_history)

[]
